In [58]:
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import keras.losses

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
#load data 

file=open("frankanstein-2.txt").read()


In [13]:
def tokenize_words(input):
    input= input.lower()
    tokenizer= RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(input)
    filtered=filter(lambda token: token not in stopwords.words('english'),tokens)
    return "".join(filtered)
processed_inputs=tokenize_words(file)

In [14]:
chars=sorted(list(set(processed_inputs)))
char_to_num=dict((c,i) for i,c in enumerate(chars))

In [16]:
input_len= len(processed_inputs)
vocab_len= len(chars)
print("Total number of characters:",input_len)
print()

Total number of characters: 233296
Total vocab: 42


In [27]:
seq_length=100
x_data=[]
y_data=[]


In [34]:
for i in range(0, input_len-seg_length,1):
    in_seq=processed_inputs[i:i + seq_length]
    out_seq=processed_inputs[i+seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns=len(x_data)
print("Total patterns:",n_patterns)

Total patterns: 233196


In [35]:
X=numpy.reshape(x_data, (n_patterns , seq_length, 1))
X=X/float(vocab_len)

In [44]:
model=Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))



In [67]:
filepath="model_weights_saved.hdf5"
checkpoint= ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,mode='min')
desired_callbacks=[checkpoint]

model.fit(X,y,epochs=4,batch_size=256,callbacks=desired_callbacks)

In [68]:
filename="model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss="categorical_crossentropy",optimizer='adam')

OSError: Unable to open file (unable to open file: name = 'model_weights_saved.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)